In [239]:
import json
import re

In [240]:
test_string='(ORDER (PIZZAORDER (NOT (TOPPING MUSHROOMS ) ) (NOT (TOPPING LIGHT ) )(COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) )  (NUMBER 1 ) (TOPPING DARK ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING ONIONS ) ) (DRINKORDER (DRINKTYPE CHERRY_COKE ) (NUMBER 4 ) (SIZE LARGE ) ) )'

In [241]:
def parse_to_dict(s):
    def parse(tokens):
        token = tokens.pop(0)
        if token == '(':
            key = tokens.pop(0)
            nested = {}
            while len(tokens) > 0 and tokens[0] != ')':
                if tokens[0] == '(':
                    temp_dict=parse(tokens)
                    for k,v in temp_dict.items():
                        if k in nested:
                            nested[k].append(v)
                        elif k == 'DRINKORDER' or k == 'PIZZAORDER' or k =='TOPPING' or k =='NOT':
                            nested[k]=[v]
                        else:
                            nested[k] = v
                else:
                    sub_key = tokens.pop(0)
                    if tokens[0] == ')':
                        tokens.pop(0)
                        return {key: sub_key}
            if len(tokens) > 0:
                tokens.pop(0)  
            return {key: nested}
        elif token == ')':
            return {}

    # Tokenize the string
    tokens = re.findall(r'\(|\)|\w+', s)
    return parse(tokens)

In [242]:
result = parse_to_dict(test_string)
print(result)

{'ORDER': {'PIZZAORDER': [{'NOT': [{'TOPPING': ['MUSHROOMS']}, {'TOPPING': ['LIGHT']}], 'COMPLEX_TOPPING': {'QUANTITY': 'extra', 'TOPPING': ['cheese']}, 'NUMBER': '1', 'TOPPING': ['DARK', 'OLIVES', 'ONIONS'], 'SIZE': 'MEDIUM'}], 'DRINKORDER': [{'DRINKTYPE': 'CHERRY_COKE', 'NUMBER': '4', 'SIZE': 'LARGE'}]}}


In [243]:
def normalize_dict_values(data):
    def normalize(data):
        if isinstance(data, dict):
            normalized_data = {}
            for key, value in data.items():
                if key in ['TOPPING', 'NOT' 'PIZZAORDER', 'DRINKORDER']:
                    normalized_data[key] = normalize(value)  # Keep these as lists
                elif isinstance(value, list) and len(value) == 1:
                    normalized_data[key] = normalize(value[0])  # Flatten single-item lists
                else:
                    normalized_data[key] = normalize(value)
            return normalized_data
        else:
            return data

    return normalize(data)

In [244]:
normalized_data = normalize_dict_values(result)
print(normalized_data)

{'ORDER': {'PIZZAORDER': {'NOT': [{'TOPPING': ['MUSHROOMS']}, {'TOPPING': ['LIGHT']}], 'COMPLEX_TOPPING': {'QUANTITY': 'extra', 'TOPPING': ['cheese']}, 'NUMBER': '1', 'TOPPING': ['DARK', 'OLIVES', 'ONIONS'], 'SIZE': 'MEDIUM'}, 'DRINKORDER': [{'DRINKTYPE': 'CHERRY_COKE', 'NUMBER': '4', 'SIZE': 'LARGE'}]}}


In [245]:
def normalize_dict_values(data):
    def normalize(data):
        if isinstance(data, dict):
            normalized_data = {}
            for key, value in data.items():
                if isinstance(value, list) and len(value) == 1 and key not in ['TOPPING', 'PIZZAORDER', 'DRINKORDER']:
                    # Convert single-item lists to direct values
                    normalized_data[key] = normalize(value[0])
                else:
                    normalized_data[key] = normalize(value)
            return normalized_data
        else:
            return data

    return normalize(data)

In [246]:
json_string = json.dumps(result)
print(json_string)

{"ORDER": {"PIZZAORDER": [{"NOT": [{"TOPPING": ["MUSHROOMS"]}, {"TOPPING": ["LIGHT"]}], "COMPLEX_TOPPING": {"QUANTITY": "extra", "TOPPING": ["cheese"]}, "NUMBER": "1", "TOPPING": ["DARK", "OLIVES", "ONIONS"], "SIZE": "MEDIUM"}], "DRINKORDER": [{"DRINKTYPE": "CHERRY_COKE", "NUMBER": "4", "SIZE": "LARGE"}]}}


In [247]:
def process_toppings(order_json):

    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        toppings = pizza.get("TOPPING", [])
        excluded = pizza.get("NOT", {})
        excluded_toppings =[]
        tops=[]
        for exc in excluded:
            tops.append(exc.get("TOPPING", []))
        for top in tops:
            for i in range(len(top)):
                excluded_toppings.append(top[i])
        if "NOT" in pizza:
            del pizza["NOT"]
        if "TOPPING" in pizza:
            del pizza["TOPPING"]
        if "COMPLEX_TOPPING" in pizza:
            del pizza["COMPLEX_TOPPING"]
        complex_topping = pizza.get("COMPLEX_TOPPING", {})
        complex_quantity = complex_topping.get("QUANTITY", None)
        complex_topping_list = complex_topping.get("TOPPING", [])
        all_toppings = []
        
        for topping in toppings:
            all_toppings.append({
                "NOT": False,
                "Quantity": None,  
                "Topping": topping
            })
        
        for topping in excluded_toppings:
            all_toppings.append({
                "NOT": True,
                "Quantity": None,  
                "Topping": topping
            })
        for topping in complex_topping_list:
            all_toppings.append({
                "NOT": False,
                "Quantity": complex_quantity,
                "Topping": topping
            })
        for drink in order["ORDER"].get("DRINKORDER", []):
            processed_order["ORDER"]["DRINKORDER"].append(drink)
        # Add the pizza order with updated AllTopping
        new_pizza_order = {**pizza, "AllTopping": all_toppings}
        processed_order["ORDER"]["PIZZAORDER"].append(new_pizza_order)
    
    return json.dumps(processed_order, indent=4)

In [248]:
processed_toppings=process_toppings(json_string)
print(processed_toppings)

{
    "ORDER": {
        "PIZZAORDER": [
            {
                "COMPLEX_TOPPING": {
                    "QUANTITY": "extra",
                    "TOPPING": [
                        "cheese"
                    ]
                },
                "NUMBER": "1",
                "SIZE": "MEDIUM",
                "AllTopping": [
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "DARK"
                    },
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "OLIVES"
                    },
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "ONIONS"
                    },
                    {
                        "NOT": true,
                        "Quantity": null,
                        "Topping": "MUSHROOMS"
          

In [249]:
def process_order(order_json):
    # Parse the input JSON
    order = json.loads(order_json)
    processed_order = {"ORDER": {"PIZZAORDER": [], "DRINKORDER": []}}
    
    for pizza in order["ORDER"].get("PIZZAORDER", []):
        number = pizza.get("NUMBER", "null")
        size = pizza.get("SIZE", "null")
        style = pizza.get("STYLE", 'null') 
        type_ = pizza.get("TYPE", 'null')  
        toppings = pizza.get("AllTopping", [])
        
        excluded_toppings = pizza.get("NOT", {}).get("TOPPING", [])
        toppings = [topping for topping in toppings if topping not in excluded_toppings]
        
        processed_order["ORDER"]["PIZZAORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "STYLE": style,
            "TYPE": type_,
            "AllTopping": toppings
        })
    for drink in order["ORDER"].get("DRINKORDER", []):
        number = drink.get("NUMBER", "null")
        size = drink.get("SIZE", "null")
        drink_type = drink.get("DRINKTYPE", "null")
        container_type = drink.get("CONTAINERTYPE", "null")

        processed_order["ORDER"]["DRINKORDER"].append({
            "NUMBER": number,
            "SIZE": size,
            "DRINKTYPE": drink_type,
            "CONTAINERTYPE": container_type
        })
    
    return json.dumps(processed_order, indent=4)

In [250]:
output_json = process_order(processed_toppings)
print(output_json)

{
    "ORDER": {
        "PIZZAORDER": [
            {
                "NUMBER": "1",
                "SIZE": "MEDIUM",
                "STYLE": "null",
                "TYPE": "null",
                "AllTopping": [
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "DARK"
                    },
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "OLIVES"
                    },
                    {
                        "NOT": false,
                        "Quantity": null,
                        "Topping": "ONIONS"
                    },
                    {
                        "NOT": true,
                        "Quantity": null,
                        "Topping": "MUSHROOMS"
                    },
                    {
                        "NOT": true,
                        "Quantity": null,
      